#### *Author: Thomas Sonderegger (sonderegger@ifu.baug.ethz.ch)*

# Adding crop activities to the soil database

This noteboook adds new crop actvities to the "soilimpacts" database based on the machinery information in the Excel database.


In [1]:
import brightway2 as bw
import pandas as pd
from mypackages import bw2_tomfuncs as tf

In [2]:
bw.projects.set_current('brightway-soil')

In [3]:
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	cutoff35
	soilimpacts

In [4]:
db = bw.Database('cutoff35')
bio = bw.Database('biosphere3')
db_soil = bw.Database('soilimpacts')

The new fieldwork processes are already in the database:

In [5]:
[i for i in db_soil]

['tillage, currying, by weeder, S' (hectare, GLO, None),
 'fertilising, by broadcaster, S' (hectare, GLO, None),
 'tillage, cultivating, chiselling, S' (hectare, GLO, None),
 'harvesting, transfer trailer, L' (unit, GLO, None),
 'sowing, M' (hectare, GLO, None),
 'tillage, rolling, M' (hectare, GLO, None),
 'tillage, harrowing, by spring tine harrow, S' (hectare, GLO, None),
 'harvesting, combine harvesting, M' (hectare, GLO, None),
 'tillage, currying, by weeder, L' (hectare, GLO, None),
 'tillage, ploughing, L' (hectare, GLO, None),
 'application of plant protection product, by field sprayer, S' (hectare, GLO, None),
 'tillage, rolling, S' (hectare, GLO, None),
 'tillage, harrowing, by rotary harrow, L' (hectare, GLO, None),
 'tillage, currying, by weeder, M' (hectare, GLO, None),
 'fertilising, by broadcaster, M' (hectare, GLO, None),
 'sowing, L' (hectare, GLO, None),
 'tillage, rolling, L' (hectare, GLO, None),
 'harvesting, transfer trailer, M' (unit, GLO, None),
 'harvesting, co

Create production process from copy:

In [6]:
def replace_exchanges(na, fw_processses, activity_processes):
    
    """
    This function ...
    """
    
    classification = 'agricultural means of production/work processes'
    omit_list = ['drying', 'grading', 'irrigation', 'milking', 'treatment']
    
    markets = [exc for exc in na.technosphere() if classification in str(exc.input['classifications'])]    
    for omit_name in omit_list:
        markets = [exc for exc in markets if omit_name not in str(exc)]


    fw_processes_activity = list()
    for p in fw_processes:
        for m in markets:
            if p in str(m):
                fw_processes_activity.append(p)
                
    for p in fw_processes_activity:
        market = [exc.input for exc in markets if p in str(exc.input)]
        amount = [exc.amount for exc in markets if p in str(exc.input)]
        exc = [exc for exc in activity_processes if p in str(exc)]
        exc = db_soil.get(exc[0])
        new_exc = na.new_exchange(input=exc, amount=amount[0], type='technosphere')
        new_exc.save()
        
    keys = [exc.input.key[1] for exc in markets]
    [exc.delete() for exc in na.technosphere() if exc.input.key[1] in keys]
    

In [7]:
def replace_amounts(na, process, amount):
    
    """
    This function ...
    """

    exc = db_soil.get(p)
    [e.delete() for e in na.technosphere() if exc.key[1] in str(e)]
    new_exc = na.new_exchange(input=exc, amount=amount, type='technosphere')
    new_exc.save()

In [8]:
def delete_exchange(na, process):
    
    """
    This function ...
    """

    exc = db_soil.get(p)
    [e.delete() for e in na.technosphere() if exc.key[1] in str(e)]
    new_exc.save()

In [9]:
def check_exchanges(na):
    
    """
    This function ...
    """
    
    classification = 'agricultural means of production/work processes'
    omit_list = ['drying', 'grading', 'irrigation', 'milking', 'treatment']
    
    markets = [exc for exc in na.technosphere() if classification in str(exc.input['classifications'])]    
    for omit_name in omit_list:
        markets = [exc for exc in markets if omit_name not in str(exc)]


    return(markets)
    

In [10]:
fw_processes = ['application of plant protection product, by field sprayer',
                'combine harvesting',
                'fertilising, by broadcaster',
                'sowing',
                'tillage, cultivating, chiselling',
                'tillage, currying, by weeder',
                'tillage, harrowing, by rotary harrow',
                'tillage, harrowing, by spring tine harrow',
                'tillage, ploughing',
                'tillage, rolling']

Create wheat processes:

In [11]:
[i.delete() for i in db_soil if 'wheat' in str(i)]

[]

Small:

In [12]:
act = db.search('wheat grain production FR')[0]
act

'wheat production' (kilogram, FR, None)

In [13]:
na = act.copy()

In [14]:
na['database'] = 'soilimpacts'
na['comment'] = "This dataset is a copy of the ecoinvent dataset 'wheat production'\
                 (kilogram, FR, None)"
na['name'] = 'wheat production, GLO, small'
na['unit'] = 'kilogram'
na['location'] = 'GLO'
na['classifications'] = [('ISIC rev.4 ecoinvent', '0111:Growing of cereals (except rice), leguminous crops and oil seeds')]
na.save()

Successfully switch activity dataset to database `soilimpacts`


In [15]:
activity_processes = ['tillage, ploughing, S',
                      'tillage, harrowing, by spring tine harrow, S',
                      'tillage, harrowing, by rotary harrow, S',
                      'tillage, cultivating, chiselling, S',
                      'tillage, rolling, S',
                      'sowing, S',
                      'fertilising, by broadcaster, S',
                      'application of plant protection product, by field sprayer, S',
                      'harvesting, combine harvesting, S']

In [16]:
replace_exchanges(na, fw_processes, activity_processes)

In [17]:
exc = db_soil.search('transfer')[0]
new_exc = na.new_exchange(input=exc, amount=1, type='technosphere')
new_exc.save()

In [18]:
check_exchanges(na)

[Exchange: 0.00102 hectare 'application of plant protection product, by field sprayer, S' (hectare, GLO, None) to 'wheat production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.00014809 hectare 'harvesting, combine harvesting, S' (hectare, GLO, None) to 'wheat production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.00051831 hectare 'fertilising, by broadcaster, S' (hectare, GLO, None) to 'wheat production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.00014809 hectare 'sowing, S' (hectare, GLO, None) to 'wheat production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 7.0463e-05 hectare 'tillage, harrowing, by rotary harrow, S' (hectare, GLO, None) to 'wheat production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.00025807 hectare 'tillage, harrowing, by spring tine harrow, S' (hectare, GLO, None) to 'wheat production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 7.7625e-05 hectare 'tillage, ploughing, S' (hectare, GLO, None) to 'wheat production, GLO, small' (kilogr

Medium:

In [19]:
act = db.search('wheat grain production FR')[0]
act

'wheat production' (kilogram, FR, None)

In [20]:
na = act.copy()

In [21]:
na['database'] = 'soilimpacts'
na['comment'] = "This dataset is a copy of the ecoinvent dataset 'wheat production'\
                 (kilogram, FR, None)"
na['name'] = 'wheat production, GLO, medium'
na['unit'] = 'kilogram'
na['location'] = 'GLO'
na['classifications'] = [('ISIC rev.4 ecoinvent', '0111:Growing of cereals (except rice), leguminous crops and oil seeds')]
na.save()

Successfully switch activity dataset to database `soilimpacts`


In [22]:
activity_processes = ['tillage, ploughing, M',
                      'tillage, harrowing, by spring tine harrow, M',
                      'tillage, harrowing, by rotary harrow, M',
                      'tillage, cultivating, chiselling, M',
                      'tillage, rolling, M',
                      'sowing, M',
                      'fertilising, by broadcaster, M',
                      'application of plant protection product, by field sprayer, M',
                      'harvesting, combine harvesting, M']

In [23]:
replace_exchanges(na, fw_processes, activity_processes)

In [24]:
exc = db_soil.search('transfer')[1]
new_exc = na.new_exchange(input=exc, amount=1, type='technosphere')
new_exc.save()

In [25]:
check_exchanges(na)

[Exchange: 0.00102 hectare 'application of plant protection product, by field sprayer, M' (hectare, GLO, None) to 'wheat production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.00014809 hectare 'harvesting, combine harvesting, M' (hectare, GLO, None) to 'wheat production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.00051831 hectare 'fertilising, by broadcaster, M' (hectare, GLO, None) to 'wheat production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.00014809 hectare 'sowing, M' (hectare, GLO, None) to 'wheat production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 7.0463e-05 hectare 'tillage, harrowing, by rotary harrow, M' (hectare, GLO, None) to 'wheat production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.00025807 hectare 'tillage, harrowing, by spring tine harrow, M' (hectare, GLO, None) to 'wheat production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 7.7625e-05 hectare 'tillage, ploughing, M' (hectare, GLO, None) to 'wheat production, GLO, medium' 

Large:

In [26]:
act = db.search('wheat grain production FR')[0]
act

'wheat production' (kilogram, FR, None)

In [27]:
na = act.copy()

In [28]:
na['database'] = 'soilimpacts'
na['comment'] = "This dataset is a copy of the ecoinvent dataset 'wheat production'\
                 (kilogram, FR, None)"
na['name'] = 'wheat production, GLO, large'
na['unit'] = 'kilogram'
na['location'] = 'GLO'
na['classifications'] = [('ISIC rev.4 ecoinvent', '0111:Growing of cereals (except rice), leguminous crops and oil seeds')]
na.save()

Successfully switch activity dataset to database `soilimpacts`


In [29]:
activity_processes = ['tillage, ploughing, L',
                      'tillage, harrowing, by spring tine harrow, L',
                      'tillage, harrowing, by rotary harrow, L',
                      'tillage, cultivating, chiselling, L',
                      'tillage, rolling, L',
                      'sowing, L',
                      'fertilising, by broadcaster, L',
                      'application of plant protection product, by field sprayer, L',
                      'harvesting, combine harvesting, L']

In [30]:
replace_exchanges(na, fw_processes, activity_processes)

In [31]:
exc = db_soil.search('transfer')[2]
new_exc = na.new_exchange(input=exc, amount=1, type='technosphere')
new_exc.save()

In [32]:
check_exchanges(na)

[Exchange: 0.00102 hectare 'application of plant protection product, by field sprayer, L' (hectare, GLO, None) to 'wheat production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.00014809 hectare 'harvesting, combine harvesting, L' (hectare, GLO, None) to 'wheat production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.00051831 hectare 'fertilising, by broadcaster, L' (hectare, GLO, None) to 'wheat production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.00014809 hectare 'sowing, L' (hectare, GLO, None) to 'wheat production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 7.0463e-05 hectare 'tillage, harrowing, by rotary harrow, L' (hectare, GLO, None) to 'wheat production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.00025807 hectare 'tillage, harrowing, by spring tine harrow, L' (hectare, GLO, None) to 'wheat production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 7.7625e-05 hectare 'tillage, ploughing, L' (hectare, GLO, None) to 'wheat production, GLO, large' (kilogr

Create maize processes:

In [33]:
[i.delete() for i in db_soil if 'maize' in str(i)]

[]

Small:

In [34]:
act = db.search('maize grain production RoW')[0]
act

'maize grain production' (kilogram, RoW, None)

In [35]:
na = act.copy()

In [36]:
na['database'] = 'soilimpacts'
na['comment'] = "This dataset is a copy of the ecoinvent dataset 'maize grain production'\
                 (kilogram, RoW, None) for which machinery has been adapted"
na['name'] = 'maize grain production, GLO, small'
na['unit'] = 'kilogram'
na['location'] = 'GLO'
na['classifications'] = [('ISIC rev.4 ecoinvent', '0111:Growing of cereals (except rice), leguminous crops and oil seeds')]
na.save()

Successfully switch activity dataset to database `soilimpacts`


In [37]:
activity_processes = ['tillage, ploughing, S',
                      'tillage, harrowing, by spring tine harrow, S',
                      'tillage, cultivating, chiselling, S',
                      'tillage, currying, by weeder, S',
                      'sowing, S',
                      'fertilising, by broadcaster, S',
                      'application of plant protection product, by field sprayer, S',
                      'harvesting, combine harvesting, S']

In [38]:
replace_exchanges(na, fw_processes, activity_processes)

In [39]:
exc = db_soil.search('transfer')[0]
new_exc = na.new_exchange(input=exc, amount=1, type='technosphere')
new_exc.save()

Set exchange amounts of fieldwork processes to 1:

In [40]:
for a in activity_processes:
    act = [exc.input for exc in na.technosphere() if a in str(exc)][0]
    [exc.delete() for exc in na.technosphere() if a in str(exc)]
    new_amount = na.new_exchange(input=act, amount=1/9315, type='technosphere')
    new_amount.save()

Adjust amounts:

In [41]:
p = 'tillage, cultivating, chiselling, S'

In [42]:
delete_exchange(na, p)

In [43]:
p = 'tillage, currying, by weeder, S'

In [44]:
delete_exchange(na, p)

In [45]:
p = 'tillage, harrowing, by spring tine harrow, S'
a = 1/9315*2

In [46]:
replace_amounts(na, p, a)

In [47]:
check_exchanges(na)

[Exchange: 1 unit 'harvesting, transfer trailer, M' (unit, GLO, None) to 'maize grain production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'tillage, ploughing, S' (hectare, GLO, None) to 'maize grain production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'sowing, S' (hectare, GLO, None) to 'maize grain production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'fertilising, by broadcaster, S' (hectare, GLO, None) to 'maize grain production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'application of plant protection product, by field sprayer, S' (hectare, GLO, None) to 'maize grain production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'harvesting, combine harvesting, S' (hectare, GLO, None) to 'maize grain production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.00021470746108427267 hectare 'tillage, harrowing, 

Medium:

In [48]:
act = db.search('maize grain production RoW')[0]
act

'maize grain production' (kilogram, RoW, None)

In [49]:
na = act.copy()

In [50]:
na['database'] = 'soilimpacts'
na['comment'] = "This dataset is a copy of the ecoinvent dataset 'maize grain production'\
                 (kilogram, RoW, None) for which machinery has been adapted"
na['name'] = 'maize grain production, GLO, medium'
na['unit'] = 'kilogram'
na['location'] = 'GLO'
na['classifications'] = [('ISIC rev.4 ecoinvent', '0111:Growing of cereals (except rice), leguminous crops and oil seeds')]
na.save()

Successfully switch activity dataset to database `soilimpacts`


In [51]:
activity_processes = ['tillage, ploughing, M',
                      'tillage, harrowing, by spring tine harrow, M',
                      'tillage, cultivating, chiselling, M',
                      'tillage, currying, by weeder, M',
                      'sowing, M',
                      'fertilising, by broadcaster, M',
                      'application of plant protection product, by field sprayer, M',
                      'harvesting, combine harvesting, M']

In [52]:
replace_exchanges(na, fw_processes, activity_processes)

In [53]:
exc = db_soil.search('transfer')[1]
new_exc = na.new_exchange(input=exc, amount=1, type='technosphere')
new_exc.save()

In [54]:
for a in activity_processes:
    act = [exc.input for exc in na.technosphere() if a in str(exc)][0]
    [exc.delete() for exc in na.technosphere() if a in str(exc)]
    new_amount = na.new_exchange(input=act, amount=1/9315, type='technosphere')
    new_amount.save()

In [55]:
p = 'tillage, cultivating, chiselling, M'

In [56]:
delete_exchange(na, p)

In [57]:
p = 'tillage, currying, by weeder, M'

In [58]:
delete_exchange(na, p)

In [59]:
p = 'tillage, harrowing, by spring tine harrow, M'
a = 1/9315*2

In [60]:
replace_amounts(na, p, a)

In [61]:
check_exchanges(na)

[Exchange: 1 unit 'harvesting, transfer trailer, L' (unit, GLO, None) to 'maize grain production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'tillage, ploughing, M' (hectare, GLO, None) to 'maize grain production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'sowing, M' (hectare, GLO, None) to 'maize grain production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'fertilising, by broadcaster, M' (hectare, GLO, None) to 'maize grain production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'application of plant protection product, by field sprayer, M' (hectare, GLO, None) to 'maize grain production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'harvesting, combine harvesting, M' (hectare, GLO, None) to 'maize grain production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.00021470746108427267 hectare 'tillage, harro

Large:

In [62]:
act = db.search('maize grain production RoW')[0]
act

'maize grain production' (kilogram, RoW, None)

In [63]:
na = act.copy()

In [64]:
na['database'] = 'soilimpacts'
na['comment'] = "This dataset is a copy of the ecoinvent dataset 'maize grain production'\
                 (kilogram, RoW, None) for which machinery has been adapted"
na['name'] = 'maize grain production, GLO, large'
na['unit'] = 'kilogram'
na['location'] = 'GLO'
na['classifications'] = [('ISIC rev.4 ecoinvent', '0111:Growing of cereals (except rice), leguminous crops and oil seeds')]
na.save()

Successfully switch activity dataset to database `soilimpacts`


In [65]:
activity_processes = ['tillage, ploughing, L',
                      'tillage, harrowing, by spring tine harrow, L',
                      'tillage, cultivating, chiselling, L',
                      'tillage, currying, by weeder, L',
                      'sowing, L',
                      'fertilising, by broadcaster, L',
                      'application of plant protection product, by field sprayer, L',
                      'harvesting, combine harvesting, L']

In [66]:
replace_exchanges(na, fw_processes, activity_processes)

In [67]:
exc = db_soil.search('transfer')[2]
new_exc = na.new_exchange(input=exc, amount=1, type='technosphere')
new_exc.save()

In [68]:
for a in activity_processes:
    act = [exc.input for exc in na.technosphere() if a in str(exc)][0]
    [exc.delete() for exc in na.technosphere() if a in str(exc)]
    new_amount = na.new_exchange(input=act, amount=1/9315, type='technosphere')
    new_amount.save()

In [69]:
p = 'tillage, cultivating, chiselling, L'

In [70]:
delete_exchange(na, p)

In [71]:
p = 'tillage, currying, by weeder, L'

In [72]:
delete_exchange(na, p)

In [73]:
p = 'tillage, harrowing, by spring tine harrow, L'
a = 1/9315*2

In [74]:
replace_amounts(na, p, a)

In [75]:
check_exchanges(na)

[Exchange: 1 unit 'harvesting, transfer trailer, S' (unit, GLO, None) to 'maize grain production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'tillage, ploughing, L' (hectare, GLO, None) to 'maize grain production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'sowing, L' (hectare, GLO, None) to 'maize grain production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'fertilising, by broadcaster, L' (hectare, GLO, None) to 'maize grain production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'application of plant protection product, by field sprayer, L' (hectare, GLO, None) to 'maize grain production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.00010735373054213633 hectare 'harvesting, combine harvesting, L' (hectare, GLO, None) to 'maize grain production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.00021470746108427267 hectare 'tillage, harrowing, 

Create soybean processes:

In [76]:
[i.delete() for i in db_soil if 'soybean' in str(i)]

[]

Small:

In [77]:
act = db.search('soybean production RoW')[0]
act

'soybean production' (kilogram, RoW, None)

In [78]:
na = act.copy()

In [79]:
na['database'] = 'soilimpacts'
na['comment'] = "This dataset is a copy of the ecoinvent dataset 'soybean production'\
                 (kilogram, RoW, None) for which machinery has been adapted"
na['name'] = 'soybean production, GLO, small'
na['unit'] = 'kilogram'
na['location'] = 'GLO'
na['classifications'] = [('ISIC rev.4 ecoinvent', '0111:Growing of cereals (except rice), leguminous crops and oil seeds')]
na.save()

Successfully switch activity dataset to database `soilimpacts`


In [80]:
activity_processes = ['tillage, ploughing, S',
                      'tillage, harrowing, by spring tine harrow, S',
                      'sowing, S',
                      'fertilising, by broadcaster, S',
                      'application of plant protection product, by field sprayer, S',
                      'harvesting, combine harvesting, S']

In [81]:
replace_exchanges(na, fw_processes, activity_processes)

In [82]:
exc = db_soil.search('transfer')[0]
new_exc = na.new_exchange(input=exc, amount=1, type='technosphere')
new_exc.save()

In [83]:
for a in activity_processes:
    act = [exc.input for exc in na.technosphere() if a in str(exc)][0]
    [exc.delete() for exc in na.technosphere() if a in str(exc)]
    new_amount = na.new_exchange(input=act, amount=1/2600, type='technosphere')
    new_amount.save()

In [84]:
p = 'tillage, ploughing, S'
a = 1/2600/4

In [85]:
replace_amounts(na, p, a)

In [86]:
check_exchanges(na)

[Exchange: 1 unit 'harvesting, transfer trailer, M' (unit, GLO, None) to 'soybean production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'tillage, harrowing, by spring tine harrow, S' (hectare, GLO, None) to 'soybean production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'sowing, S' (hectare, GLO, None) to 'soybean production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'fertilising, by broadcaster, S' (hectare, GLO, None) to 'soybean production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'application of plant protection product, by field sprayer, S' (hectare, GLO, None) to 'soybean production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'harvesting, combine harvesting, S' (hectare, GLO, None) to 'soybean production, GLO, small' (kilogram, GLO, None)>,
 Exchange: 9.615384615384615e-05 hectare 'tillage, ploughing, S' (hec

Medium:

In [87]:
act = db.search('soybean production RoW')[0]
act

'soybean production' (kilogram, RoW, None)

In [88]:
na = act.copy()

In [89]:
na['database'] = 'soilimpacts'
na['comment'] = "This dataset is a copy of the ecoinvent dataset 'soybean production'\
                 (kilogram, RoW, None) for which machinery has been adapted"
na['name'] = 'soybean production, GLO, medium'
na['unit'] = 'kilogram'
na['location'] = 'GLO'
na['classifications'] = [('ISIC rev.4 ecoinvent', '0111:Growing of cereals (except rice), leguminous crops and oil seeds')]
na.save()

Successfully switch activity dataset to database `soilimpacts`


In [90]:
activity_processes = ['tillage, ploughing, M',
                      'tillage, harrowing, by spring tine harrow, M',
                      'sowing, M',
                      'fertilising, by broadcaster, M',
                      'application of plant protection product, by field sprayer, M',
                      'harvesting, combine harvesting, M']

In [91]:
replace_exchanges(na, fw_processes, activity_processes)

In [92]:
exc = db_soil.search('transfer')[1]
new_exc = na.new_exchange(input=exc, amount=1, type='technosphere')
new_exc.save()

In [93]:
for a in activity_processes:
    act = [exc.input for exc in na.technosphere() if a in str(exc)][0]
    [exc.delete() for exc in na.technosphere() if a in str(exc)]
    new_amount = na.new_exchange(input=act, amount=1/2600, type='technosphere')
    new_amount.save()

In [94]:
p = 'tillage, ploughing, M'
a = 1/2600/4

In [95]:
replace_amounts(na, p, a)

In [96]:
check_exchanges(na)

[Exchange: 1 unit 'harvesting, transfer trailer, L' (unit, GLO, None) to 'soybean production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'tillage, harrowing, by spring tine harrow, M' (hectare, GLO, None) to 'soybean production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'sowing, M' (hectare, GLO, None) to 'soybean production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'fertilising, by broadcaster, M' (hectare, GLO, None) to 'soybean production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'application of plant protection product, by field sprayer, M' (hectare, GLO, None) to 'soybean production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'harvesting, combine harvesting, M' (hectare, GLO, None) to 'soybean production, GLO, medium' (kilogram, GLO, None)>,
 Exchange: 9.615384615384615e-05 hectare 'tillage, ploughing, M

Large:

In [97]:
act = db.search('soybean production RoW')[0]
act

'soybean production' (kilogram, RoW, None)

In [98]:
na = act.copy()

In [99]:
na['database'] = 'soilimpacts'
na['comment'] = "This dataset is a copy of the ecoinvent dataset 'soybean production'\
                 (kilogram, RoW, None) for which machinery has been adapted"
na['name'] = 'soybean production, GLO, large'
na['unit'] = 'kilogram'
na['location'] = 'GLO'
na['classifications'] = [('ISIC rev.4 ecoinvent', '0111:Growing of cereals (except rice), leguminous crops and oil seeds')]
na.save()

Successfully switch activity dataset to database `soilimpacts`


In [100]:
activity_processes = ['tillage, ploughing, L',
                      'tillage, harrowing, by spring tine harrow, L',
                      'sowing, M',
                      'fertilising, by broadcaster, L',
                      'application of plant protection product, by field sprayer, L',
                      'harvesting, combine harvesting, L']

In [101]:
replace_exchanges(na, fw_processes, activity_processes)

In [102]:
exc = db_soil.search('transfer')[2]
new_exc = na.new_exchange(input=exc, amount=1, type='technosphere')
new_exc.save()

In [103]:
for a in activity_processes:
    act = [exc.input for exc in na.technosphere() if a in str(exc)][0]
    [exc.delete() for exc in na.technosphere() if a in str(exc)]
    new_amount = na.new_exchange(input=act, amount=1/2600, type='technosphere')
    new_amount.save()

In [104]:
p = 'tillage, ploughing, L'
a = 1/2600/4

In [105]:
replace_amounts(na, p, a)

In [106]:
check_exchanges(na)

[Exchange: 1 unit 'harvesting, transfer trailer, S' (unit, GLO, None) to 'soybean production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'tillage, harrowing, by spring tine harrow, L' (hectare, GLO, None) to 'soybean production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'sowing, M' (hectare, GLO, None) to 'soybean production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'fertilising, by broadcaster, L' (hectare, GLO, None) to 'soybean production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'application of plant protection product, by field sprayer, L' (hectare, GLO, None) to 'soybean production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 0.0003846153846153846 hectare 'harvesting, combine harvesting, L' (hectare, GLO, None) to 'soybean production, GLO, large' (kilogram, GLO, None)>,
 Exchange: 9.615384615384615e-05 hectare 'tillage, ploughing, L' (hec

In [107]:
db_soil.search('soybean production')

['soybean production, GLO, large' (kilogram, GLO, None),
 'soybean production, GLO, small' (kilogram, GLO, None),
 'soybean production, GLO, medium' (kilogram, GLO, None)]

In [108]:
[exc.input for exc in db_soil.search('soybean production')[0].technosphere()]

['market for organophosphorus-compound, unspecified' (kilogram, GLO, None),
 'market for dinitroaniline-compound' (kilogram, GLO, None),
 'market for benzimidazole-compound' (kilogram, GLO, None),
 'market for lime' (kilogram, RER, None),
 'market for benzo[thia]diazole-compound' (kilogram, GLO, None),
 'market for urea, as N' (kilogram, GLO, None),
 'market for potassium chloride, as K2O' (kilogram, GLO, None),
 'market for ammonium nitrate, as N' (kilogram, GLO, None),
 'market for phosphate rock, as P2O5, beneficiated, dry' (kilogram, GLO, None),
 'market for soybean seed, for sowing' (kilogram, GLO, None),
 'market for [sulfonyl]urea-compound' (kilogram, GLO, None),
 'market for [thio]carbamate-compound' (kilogram, GLO, None),
 'market for phosphate fertiliser, as P2O5' (kilogram, GLO, None),
 'market for acetamide-anillide-compound, unspecified' (kilogram, GLO, None),
 'market for pyrethroid-compound' (kilogram, GLO, None),
 'market for ammonia, liquid' (kilogram, RER, None),
 'ma

In [109]:
[exc.input for exc in db_soil.search('soybean production')[1].technosphere()]

['market for organophosphorus-compound, unspecified' (kilogram, GLO, None),
 'market for dinitroaniline-compound' (kilogram, GLO, None),
 'market for benzimidazole-compound' (kilogram, GLO, None),
 'market for lime' (kilogram, RER, None),
 'market for benzo[thia]diazole-compound' (kilogram, GLO, None),
 'market for urea, as N' (kilogram, GLO, None),
 'market for potassium chloride, as K2O' (kilogram, GLO, None),
 'market for ammonium nitrate, as N' (kilogram, GLO, None),
 'market for phosphate rock, as P2O5, beneficiated, dry' (kilogram, GLO, None),
 'market for soybean seed, for sowing' (kilogram, GLO, None),
 'market for [sulfonyl]urea-compound' (kilogram, GLO, None),
 'market for [thio]carbamate-compound' (kilogram, GLO, None),
 'market for phosphate fertiliser, as P2O5' (kilogram, GLO, None),
 'market for acetamide-anillide-compound, unspecified' (kilogram, GLO, None),
 'market for pyrethroid-compound' (kilogram, GLO, None),
 'market for ammonia, liquid' (kilogram, RER, None),
 'ma

In [110]:
[exc.input for exc in db_soil.search('soybean production')[2].technosphere()]

['market for organophosphorus-compound, unspecified' (kilogram, GLO, None),
 'market for dinitroaniline-compound' (kilogram, GLO, None),
 'market for benzimidazole-compound' (kilogram, GLO, None),
 'market for lime' (kilogram, RER, None),
 'market for benzo[thia]diazole-compound' (kilogram, GLO, None),
 'market for urea, as N' (kilogram, GLO, None),
 'market for potassium chloride, as K2O' (kilogram, GLO, None),
 'market for ammonium nitrate, as N' (kilogram, GLO, None),
 'market for phosphate rock, as P2O5, beneficiated, dry' (kilogram, GLO, None),
 'market for soybean seed, for sowing' (kilogram, GLO, None),
 'market for [sulfonyl]urea-compound' (kilogram, GLO, None),
 'market for [thio]carbamate-compound' (kilogram, GLO, None),
 'market for phosphate fertiliser, as P2O5' (kilogram, GLO, None),
 'market for acetamide-anillide-compound, unspecified' (kilogram, GLO, None),
 'market for pyrethroid-compound' (kilogram, GLO, None),
 'market for ammonia, liquid' (kilogram, RER, None),
 'ma

In [111]:
db_soil.search('transfer')[0]

'harvesting, transfer trailer, M' (unit, GLO, None)